In [42]:
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
import xgboost as xgb
import elm
from imblearn.over_sampling import SMOTE 

import dataloader
import bandpower

# Leave-one-subject out cross validation

In [50]:
label_format = 3
X, Y, df_all = dataloader.read_data(label_format=label_format)

# Transform into power
low, high = list(range(1,50)), list(range(2,51))
powers = bandpower.get_bandpower(X, 500, low=low, high=high, dB_scale=True)

Load data from .mat files...


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:495: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val = np.array(val, copy=False)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Calculating the bandpower of time-series data...
freqs:  [0.000e+00 2.000e-01 4.000e-01 ... 2.496e+02 2.498e+02 2.500e+02]


In [51]:
# Select models
model_names = ['svm', 'xgboost', 'elm']
sampling_names = ['original', 'SMOTE']
cv_modes = ['KFold', 'LOSO']

for model_name, sampling_name, cv_mode in itertools.product(model_names, sampling_names, cv_modes):

    file_name = 'label%d_%s_%s_%s.csv'%(label_format, model_name, sampling_name, cv_mode)
    description = 'Label: %d, Model: %s, Sampling: %s, CV mode: %s'%(label_format, model_name, sampling_name, cv_mode)

    # Average band power over channels
    num_sample = len(X)
    X_power = np.zeros((num_sample, len(low)))
    for i in range(len(X)):
        X_power[i,:] = np.mean(powers[i],0)

    kf = KFold(n_splits=10, shuffle=True, random_state=23)
    S = df_all['subject'].values
    folds = kf.split(X_power) if cv_mode=='KFold' else np.unique(S)

    train_acc_list = []
    val_acc_list = []
    num_val_list = []
    chance_list = []
    fold_name_list = []

    print('Sub  \t Chance\t | Train | Val')
    for i_fold, fold in enumerate(folds):
        if cv_mode == 'KFold':
            train_indices = fold[0]
            val_indices = fold[1]
        else:
            subID = fold
            train_indices = np.where(S!=subID)[0]
            val_indices = np.where(S==subID)[0]

        X_train, Y_train = X_power[train_indices,...], Y[train_indices]
        X_val, Y_val = X_power[val_indices,...], Y[val_indices]

        # Resample training data
        if sampling_name == 'SMOTE':
            sm = SMOTE(random_state=23)
            X_train, Y_train = sm.fit_resample(X_train, Y_train)

        # Train classifier
        if model_name in ['svm','xgboost']:
            if model_name == 'svm':
                clf = svm.SVC()
            elif model_name == 'xgboost':
                clf = xgb.XGBClassifier()
            clf.fit(X_train, Y_train)

            # Test classifier
            train_acc = clf.score(X_train, Y_train)
            val_acc = clf.score(X_val, Y_val)
        elif model_name == 'elm':
            clf = elm.ELMKernel()
            train_elm_data = np.concatenate((Y_train[:,np.newaxis], X_train), axis=1)
            val_elm_data = np.concatenate((Y_val[:,np.newaxis], X_val), axis=1)
            clf.search_param(train_elm_data, cv="kfold", of="accuracy", eval=10)
            train_acc = clf.train(train_elm_data).get_accuracy()
            val_acc = clf.test(val_elm_data).get_accuracy()

        fold_name = 'Fold %d'%(i_fold) if cv_mode == 'KFold' else 'Sub %d'%(subID)
        chance = np.sum(Y_train)/len(Y_train)
        chance = (1-chance) if chance<0.5 else chance

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        num_val_list.append(len(Y_val))
        fold_name_list.append(fold_name)
        chance_list.append(chance)

        print('%s\t %.1f%%\t | %.1f%% | %.1f%%'%(fold_name, chance*100, train_acc*100, val_acc*100))

    avg_acc = sum([val_acc_list[i]*num_val_list[i] for i in range(len(num_val_list))])/num_sample
    print('Average val acc: %.1f%%'%(avg_acc*100))

    # Save result as csv file
    df_result = pd.DataFrame({description: fold_name_list, 'Chance': chance_list, 
                              'Train': train_acc_list, 'Val': val_acc_list})
    df_result.loc[len(df_result),description] = 'Average val acc: %.1f%%'%(avg_acc*100)
    df_result.to_csv('./results/%s'%(file_name))

Sub  	 Chance	 | Train | Val
Fold 0	 80.3%	 | 80.3% | 76.9%
Fold 1	 79.5%	 | 79.5% | 84.6%
Fold 2	 79.5%	 | 79.5% | 84.6%
Fold 3	 81.2%	 | 81.2% | 69.2%
Fold 4	 80.3%	 | 80.3% | 76.9%
Fold 5	 78.6%	 | 78.6% | 92.3%
Fold 6	 79.5%	 | 79.5% | 84.6%
Fold 7	 81.2%	 | 81.2% | 69.2%
Fold 8	 79.5%	 | 79.5% | 84.6%
Fold 9	 80.3%	 | 80.3% | 76.9%
Average val acc: 80.0%
Sub  	 Chance	 | Train | Val
Sub 1	 80.0%	 | 80.0% | 80.0%
Sub 2	 79.8%	 | 79.8% | 100.0%
Sub 3	 80.6%	 | 80.6% | 66.7%
Sub 4	 79.8%	 | 79.8% | 83.3%
Sub 5	 79.0%	 | 79.0% | 100.0%
Sub 6	 81.6%	 | 81.6% | 40.0%
Sub 7	 80.0%	 | 80.0% | 80.0%
Sub 8	 79.4%	 | 79.4% | 100.0%
Sub 9	 80.5%	 | 80.5% | 50.0%
Sub 10	 79.5%	 | 79.5% | 100.0%
Sub 11	 79.8%	 | 79.8% | 83.3%
Sub 12	 82.3%	 | 82.3% | 33.3%
Sub 13	 79.2%	 | 79.2% | 100.0%
Sub 14	 79.7%	 | 79.7% | 100.0%
Sub 16	 80.6%	 | 80.6% | 0.0%
Sub 17	 80.3%	 | 80.3% | 66.7%
Sub 18	 80.2%	 | 80.2% | 75.0%
Sub 19	 80.0%	 | 80.0% | 80.0%
Sub 20	 80.5%	 | 80.5% | 75.0%
Sub 21	 78.5%	 | 78.5% |

/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Kernel function:  linear  best cv value:  0.7416666666666667
Kernel function:  poly  best cv value:  0.45555555555555555
##### Search complete #####

Regressor Parameters

Regularization coefficient:  2.2483734909356063
Kernel Function:  rbf
Kernel parameters:  [0.6232831104148938]

CV error:  0.8055555555555556

Fold 0	 80.3%	 | 100.0% | 76.9%
elmk
##### Start search #####
Kernel function:  rbf  best cv value:  0.8
Kernel function:  linear  best cv value:  0.7444444444444445
Kernel function:  poly  best cv value:  0.3555555555555555
##### Search complete #####

Regressor Parameters

Regularization coefficient:  -0.801277403007913
Kernel Function:  rbf
Kernel parameters:  [-0.5349111372992282]

CV error:  0.8

Fold 1	 79.5%	 | 79.5% | 84.6%
elmk
##### Start search #####
Kernel function:  rbf  best cv value:  0.7972222222222223
Kernel function:  linear  best cv value:  0.7666666666666666
Kernel function:  poly  best cv value:  0.35833333333333334
##### Search complete #####

Regressor P

Kernel function:  linear  best cv value:  0.7483516483516484
Kernel function:  poly  best cv value:  0.32857142857142857
##### Search complete #####

Regressor Parameters

Regularization coefficient:  -0.13489110986295721
Kernel Function:  rbf
Kernel parameters:  [0.4373265991829188]

CV error:  0.8076923076923077

Sub 11	 79.8%	 | 79.8% | 83.3%
elmk
##### Start search #####
Kernel function:  rbf  best cv value:  0.8307692307692307
Kernel function:  linear  best cv value:  0.7934065934065935
Kernel function:  poly  best cv value:  0.4879120879120879
##### Search complete #####

Regressor Parameters

Regularization coefficient:  -1.0721437982225028
Kernel Function:  rbf
Kernel parameters:  [0.06553583448723488]

CV error:  0.8307692307692307

Sub 12	 82.3%	 | 82.3% | 33.3%
elmk
##### Start search #####
Kernel function:  rbf  best cv value:  0.8
Kernel function:  linear  best cv value:  0.7442307692307693
Kernel function:  poly  best cv value:  0.7067307692307693
##### Search complete ##

/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


...

Kernel function:  rbf  best cv value:  0.595
Kernel function:  linear  best cv value:  0.7422222222222222
Kernel function:  poly  best cv value:  0.6972222222222222
##### Search complete #####

Regressor Parameters

Regularization coefficient:  -1.2341635685310886
Kernel Function:  linear
Kernel parameters:  []

CV error:  0.7422222222222222

Sub 24	 50.0%	 | 85.9% | 50.0%
elmk
##### Start search #####


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Kernel function:  rbf  best cv value:  0.58125
Kernel function:  linear  best cv value:  0.72
Kernel function:  poly  best cv value:  0.67
##### Search complete #####

Regressor Parameters

Regularization coefficient:  0.13112022233332488
Kernel Function:  linear
Kernel parameters:  []

CV error:  0.72

Sub 25	 50.0%	 | 84.7% | 33.3%
elmk
##### Start search #####


/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/hundredball/anaconda3/envs/Math24_new/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Kernel function:  rbf  best cv value:  0.5175824175824176
Kernel function:  linear  best cv value:  0.7703296703296703
Kernel function:  poly  best cv value:  0.6835164835164835
##### Search complete #####

Regressor Parameters

Regularization coefficient:  0.8433896642097217
Kernel Function:  linear
Kernel parameters:  []

CV error:  0.7703296703296703

Sub 26	 50.0%	 | 91.1% | 42.9%
Average val acc: 50.0%
